In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_score, recall_score, precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import label_binarize


ModuleNotFoundError: No module named 'tensorflow'

In [4]:

# تحميل البيانات
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# تصفية البيانات لاختيار الفئات من 0 إلى 4 فقط
mask_train = np.isin(train_labels, [0, 1, 2, 3, 4])
train_images_filtered = train_images[mask_train]
train_labels_filtered = train_labels[mask_train]

mask_test = np.isin(test_labels, [0, 1, 2, 3, 4])
test_images_filtered = test_images[mask_test]
test_labels_filtered = test_labels[mask_test]


NameError: name 'fashion_mnist' is not defined

In [ ]:

# تحويل الصور إلى مصفوفات 1D
train_images_filtered = train_images_filtered.reshape((train_images_filtered.shape[0], 28 * 28))
test_images_filtered = test_images_filtered.reshape((test_images_filtered.shape[0], 28 * 28))

# توحيد القيم لتكون بين 0 و 1
train_images_filtered, test_images_filtered = train_images_filtered / 255.0, test_images_filtered / 255.0


In [ ]:

# تدريب نموذج الانحدار اللوجستي
model = LogisticRegression(max_iter=1000)
model.fit(train_images_filtered, train_labels_filtered)


In [ ]:

# إجراء التنبؤات
predictions = model.predict(test_images_filtered)

# مصفوفة الارتباك
cm = confusion_matrix(test_labels_filtered, predictions)
print("Confusion Matrix:")
print(cm)


In [ ]:

# تقرير التصنيف
print("Classification Report:")
print(classification_report(test_labels_filtered, predictions))

# حساب الدقة (accuracy), الدقة (precision), الاسترجاع (recall) لكل فئة
accuracy = np.mean(predictions == test_labels_filtered)
precision = precision_score(test_labels_filtered, predictions, average='weighted')
recall = recall_score(test_labels_filtered, predictions, average='weighted')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")


In [ ]:

# تحويل التسميات إلى مصفوفة ثنائية (One-vs-Rest) للفئات من 0 إلى 4
y_test_bin = label_binarize(test_labels_filtered, classes=np.arange(5))

# حساب منحنى ROC لكل فئة باستخدام predict_proba
fpr = {}
tpr = {}
roc_auc = {}

for i in range(5):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], model.predict_proba(test_images_filtered)[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# رسم منحنيات ROC لجميع الفئات في نفس الرسم
plt.figure(figsize=(10, 8))
colors = ['aqua', 'darkorange', 'green', 'red', 'blue']

for i, color in zip(range(5), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'Class {i} (AUC = {roc_auc[i]:0.2f})')

# رسم الخطوط الأساسية (الخط العشوائي)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

# تسميات المحاور
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Multiple Classes')
plt.legend(loc="lower right")
plt.show()


In [ ]:

# رسم منحنى Precision-Recall لكل فئة
plt.figure(figsize=(10, 8))
for i, color in zip(range(5), colors):
    precision_vals, recall_vals, _ = precision_recall_curve(y_test_bin[:, i], model.predict_proba(test_images_filtered)[:, i])
    plt.plot(recall_vals, precision_vals, color=color, lw=2, label=f'PR curve for class {i} (Precision-Recall)')


In [ ]:

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves for Multiple Classes')
plt.legend(loc="lower left")
plt.show()


In [ ]:

# عرض مصفوفة الارتباك
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.arange(5), yticklabels=np.arange(5))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix for 5 Classes')
plt.show()


In [ ]:

# عرض بعض الصور مع التنبؤات
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(test_images_filtered[i].reshape(28, 28), cmap='gray')
    plt.title(f"True: {test_labels_filtered[i]}, Pred: {predictions[i]}")
    plt.axis('off')
plt.tight_layout()
plt.show()